# Pore Scale Imaging and Modeling Section I

In this example we calculate the permeability of a Mount Gambier rock sample. A comprehensive related paper to this rock sample is [pore scale imaging and modeling](https://www.sciencedirect.com/science/article/pii/S0309170812000528).

In [ ]:
%%html
<style>
table {float:left}
</style>

### Extracted network
In this project, we have used [SNOW algorithm](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.96.023307) in [Porespy](http://porespy.org/) which is a network extraction method based on marker-based watershed segmentation.  The following figures illustrate one segment of CT images of the rock sample in binarized version and its recunstructed image:

<img src="https://i.imgur.com/sZoo5xO.jpg" style="width: 25%" align="left"/>
<img src="https://i.imgur.com/ls3ar6c.jpg" style="width: 25%"align="left"/>

| Sample                           | Size           | Resolution | Porosity |
| :---                             | :---           |:---        |:---      |
| Mount Gambier (our model)        | 512 512 512    | 3.024  μm  | 0.436    |
| Mount Gambier (paper)            | 350 350 350    | 9  μm      | 0.556    | 

Let's import the modules and required models:

In [1]:
import porespy as ps
import matplotlib.pyplot as plt
import openpnm as op
import numpy as np
import scipy as sp
from skimage import io

In [ ]:
We extracted the network using SNOW algorithm. To save the time we load the extracted network model in this example:

In [8]:
mgr = op.Workspace()
mgr.clear()
mgr.keys()
fname = '../../fixtures/Extracted-Networks/MountGampn'
mgr.load_workspace(fname)
pn=mgr['sim_03']['net_01']
print(pn)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
openpnm.network.GenericNetwork : net_01
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Properties                                    Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.area                                      5780 / 5780 
2     pore.centroid                                  5780 / 5780 
3     pore.coords                                    5780 / 5780 
4     pore.diameter                                  5780 / 5780 
5     pore.equivalent_diameter                       5780 / 5780 
6     pore.extended_diameter                         5780 / 5780 
7     pore.inscribed_diameter                        5780 / 5780 
8     pore.label                                     5780 / 5780 
9     pore.surface_area                              5780 / 5780 
10    pore.volume                                    5780 / 5780

Add a generic geometry object to the network. The geometry parameters (pore diameter, ..) are already inclded in the extracted network.

In [10]:
proj = pn.project
coord_num_avg=np.mean(pn.num_neighbors(pores=pn.Ps))
geom = op.geometry.GenericGeometry(network=pn, pores=pn['pore.all'], throats=pn['throat.all'],project=proj)

We then define a generic phase and physics object for oil phase and assign the related physics models that are required in hydraulic flow.

In [11]:
oil = op.phases.GenericPhase(network=pn,project=proj)
oil['pore.viscosity']=0.547e-3
phys_oil = op.physics.GenericPhysics(network=pn, phase=oil, geometry=geom,project=proj)
mod = op.models.physics.hydraulic_conductance.hagen_poiseuille
phys_oil.add_model(propname='throat.hydraulic_conductance',
                              model=mod)

The network properties of our model and the sample in the paper are given in the following table. Note that the difference are related to the difference in the image samples we used (size, connections,...)

In [12]:
print('Number of pores:'+str(len(pn.Ps))+' Number of throats:'+str(len(pn.Ts))+' avg coordination number:'+str(coord_num_avg))

Number of pores:5780 Number of throats:10128 avg coordination number:3.5044982698961937


| Model | Number of pores | Number of throats | Volume (mm3) | Coordination number |
| --- | --- | --- | --- | --- |
| Mount Gambier Carbonate (512) | 5780 (4679 internal) | 10128 (9027 internal) | 27.65 | 3.504 |
| MG Paper (350) | 22665 (257 elements isolated) | 84593 | 31.3 | 7.41 |

## Permeability Calculation Algorithm

The StokesFlow class is for simulation of viscous flow. In this class default property names will be set. The main role of this class would be calculation of the hydraulic permeability. Having its effective permeability calculation method, it can deal with nonuniform medias.

As we have mentioned the permeability will be a tensor, which represents $K_x,K_y,K_z$. Permeability tensor plays an important role in anisotropic medias charactarization. We calculate permeabilities in three directions.

To find the permeability of the medium, "calc_effective_permeability" method is used. This method can estimate the length and area of the medium automatically. However,in this example we define methods to find domain length and area. These methods are called within the permeability calculation loop.

In [13]:
K_oil = [None,None,None]
bounds = [ ['top', 'bottom'], ['left', 'right'],['front', 'back']]
[amax, bmax, cmax] = np.max(pn['pore.coords'], axis=0)
[amin, bmin, cmin] = np.min(pn['pore.coords'], axis=0)
lx = amax-amin
ly = bmax-bmin
lz = cmax-cmin
da = lx*ly
dl = lz

def top_b(lx,ly,lz):
    da = lx*ly
    dl = lz
    res_2=[da,dl]
    return res_2

def left_r(lx,ly,lz):
    
    da = lx*lz
    dl = ly
    res_2=[da,dl]
    return res_2

def front_b(lx,ly,lz):
    da = ly*lz
    dl = lx
    res_2=[da,dl]
    return res_2

options = {0 : top_b(lx,ly,lz),1 : left_r(lx,ly,lz),2 : front_b(lx,ly,lz)}

for bound_increment in range(len(bounds)):
    BC1_pores = pn.pores(labels=bounds[bound_increment][0])
    BC2_pores = pn.pores(labels=bounds[bound_increment][1])
    [da,dl]=options[bound_increment]
    # Permeability - oil
    sf_oil = op.algorithms.StokesFlow(network=pn, phase=oil)
    sf_oil.setup(conductance='throat.hydraulic_conductance')
    sf_oil._set_BC(pores=BC1_pores, bctype='value', bcvalues=1000)
    sf_oil._set_BC(pores=BC2_pores, bctype='value', bcvalues=0)
    sf_oil.run()
    K_oil[bound_increment] = sf_oil.calc_effective_permeability(domain_area=da,
                                                                             domain_length=dl,
                                                                             inlets=BC1_pores,
                                                                             outlets=BC2_pores)
    proj.purge_object(obj=sf_oil)

In [14]:
print('Permeabilities in x,y,z direction:'+ str(K_oil[0])+str(K_oil[1])+str(K_oil[2])+'m^2/s')
print('avg permeability '+str(np.mean(K_oil)/1e-13)+' darcy')

Permeabilities in x,y,z direction:[1.98721524e-12][2.32942385e-12][1.75726297e-12]m^2/s
avg permeability 20.24634018983047 darcy


Results for permeability calculation of four samples are given in the following. As we see the results for Bentheimer which is a sand stone rock is very close to the value given in the paper.

|Sample paper | Mount Gambier 350 |  |
| --- | --- | --- | 
|Kavg (from image)| 19.2 |  |

#### Notes on network extraction:

1) The SNOW algorithm concludes four main steps:
- Prefiltering the distance map
- Eliminating peaks on saddles and plateaus
- Merging peaks that are too near each other
- Assigning void voxels to the appropriate pore using a marker-based watershed.

In the prefilterin step parameters of the gaussian filter in Porespy can be adjusted to reach even better segmentations. The idea have been shown in Fig.4 of the related [paper](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.96.023307). For this example we kept the parameters as default.

2) Based on new porespy package, there is also some changes in SNOW algorithm previous version. In the previous version the area was estimated as the number of voxels on the surface multiplied by the area of
one voxel face. Now the user can have the chance to use [Marching Cube](https://en.wikipedia.org/wiki/Marching_cubes) algorithm. Although this option will give better results, we can still turn it off in SNOW algorithm for the sake of time efficiency and still have good results.

#### Notes on the results:

The porosity is being calculated from the voxelated image in a similar manner of the paper.  The permeabilities have been calculated using stokes flow algorithm. The difference might be related to the error which lays behind the parameters in filtering process (sigma, R). We have used default values of sigma=0.4 and R=5 in all samples, which may lead to misrepresentation of the network.

The difference in permeability may also be related to the different conduit lengths. In the Blunt's paper they have defined a shape factor to account for the non-cylindrical deviation of the throats. This shape factor is whithin the maximal extraction method. In the SNOW algorithm, using the equivalent diameter rather than inscribed diameter for the hydraulic conductance (assumming no P loss in the pores) will provide better results in the permeability calculation.

From the Berea sandstone results in another example, we can also comment on the effect of the structures of the rock sample. For sandstones, the morphology is more ideal than carbonates for network extractions. We also get a good result for Bentheimer Sandstone permeability. But for the carbonate cases, it is different. As we see in their CT images, there are Fossil grains (Pebbles in ketton, other fossil shells in two other sample) which provide different length scales of micro to macro pores. For example it is recommended to use multiscale pore network extraction.

As long as our sample is not the same sample in the Blunt's paper (they are from the same rock but different resolution, size, and part of the rock), the slight difference in results is acceptable.